In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/beans/bespoke')

from constants import *
from imports import *

In [6]:
for i in range(100):
    paths = glob.glob(f"{BLENDER_MEMBANK_ROOT}/**/obs_per_sec.npy", recursive=True)
    obs_per_sec = np.array([np.load(p)[0] for p in paths])
    print(obs_per_sec)
    time.sleep(60*1)

[]
[]
[14.237812  10.356549  10.859507  11.556031  11.709595  13.498773
 12.7230425 11.695058  10.860791  12.911948  12.406394 ]
[13.216111  10.356549  10.859507  11.556031  11.709595  13.498773
 12.7230425 11.695058  17.656933  12.911948  10.022588  12.406394 ]
[13.216111 12.940002 12.13802  12.934618 12.616057 12.18158  12.336537
 13.687449 17.656933 13.301823 14.05113  10.211433]
[16.68072  14.012879 14.828089 12.934618 16.23838  11.564788 15.246547
 11.339076 12.483118 11.363749 14.05113  16.423649]
[13.331092 14.012879 14.828089 10.442285 14.92601  15.9981   13.932509
 11.339076 16.918922 11.363749 11.838459 16.423649]
[14.240126 11.368153 12.253025 13.126512 15.462808 15.9981   13.932509
 13.456455 17.676895 13.032362 11.225404 14.257575]
[14.240126 17.378386 20.604376 13.787663 15.462808 12.957113 11.84753
 14.444202 17.18012  12.546709 11.225404 13.154066]
[13.862181 17.378386 17.540937 13.787663 15.667718 12.957113 13.322141
 14.444202 17.18012  16.132223 14.850071 16.05152 ]


[14.477525 13.352584 11.948181 12.637859 16.92278  12.017134 11.739449
 13.813476 16.032358 15.759639 11.354961 14.219231]
[14.477525 12.380087 11.984758 14.597959 11.063392 13.149035 14.96715
 12.383003 16.032358 12.976895 15.201333 13.423804]
[12.624185  14.618938  11.984758  14.521697  11.063392  13.149035
 12.322888  12.180721  11.910913  12.976895  12.8835535 14.56279  ]
[12.624185 14.480675 13.740746 14.521697 12.860165 11.315089 12.322888
 12.180721 14.63542  14.794536 13.690215 14.56279 ]
[ 7.380163 14.917616 15.423501 11.905883 12.923255 16.275677 13.683496
 14.4656   12.502339 11.016962 13.690215 12.288526]
[ 7.380163  14.917616  13.313937  11.905883  12.923255  12.11529
 11.6504135 11.796222  12.502339  11.016962  16.292303  11.310069 ]
[14.612216  17.88608   13.313937  11.640616  13.942877  12.11529
 11.6504135 11.796222  13.855021  14.90085   12.530535  11.310069 ]
[12.162118  12.517741  12.9694    16.302063  12.832177  12.801398
 12.588312  12.263039  12.8491335 10.802132

In [5]:
set_should_stop(True)

In [12]:
set_lr(3e-4)

In [8]:
import numpy as np

In [9]:
a = np.array([1,2,3,4,1,2,3])

In [11]:
np.where(a==1)[0]

array([0, 4])

In [12]:
max(np.nan, 0)

nan

In [13]:
np.clip(12, 10,11)

11